***3D Convolutional Neural Networks using 5 convolutional layers***

**INTRODUCTION**
The project deals with the identification of lung cancer, which affects roughly about 225,000 people every year and accounts to about 12 billion in health care costs in the final stages. Therefore, early detection of cancer is critical to give the patients the best chance of recovery and survival.

The goal of this project is to evaluate the data (Slices of CT scans) provided with various pre-processing techniques and analyze the data using machine learning algorithms, in this case 3D Convolutional Neural Networks to train and validate the model, to create an accurate model which can be used to determine whether a person has cancer or not. This will greatly help in the identification and elimination of cancer cells in the early stages. Therefore, an automated method capable of determining whether the patient will be diagnosed with lung cancer is the aim of this project.

**Dataset Information**
            The dataset consists of thousands of CT images from high risk patients in DICOM format (detailed information given below). Each image contains a series with multiple axial slices of the chest cavity and a variable number of 2D slices, which can vary based on the machine taking the scan and the patient.

**DICOM Standard**
        DICOM is an acronym for Digital Imaging and Communications in Medicine. Files in this format are saved with either a DCM or DCM30 file extension, but some may not have an extension at all. It is used for both communication protocol and a file format i.e., it can store medical information, such as ultrasound and MRI images along with the patient’s information, all in one file.
          The format ensures that all the data stays together and provides the ability to transfer information between devices that support the DICOM format. To view the information in the files there are two methods, they are 
*Opening DICOM files with Free viewer*
                 I have used the MicroDicom File viewer, which is a free to use software exclusively for viewing the DICOM files. The image along with the metadata can be found making it easy and efficient to access the data for research and development purposes.
*Using Libraries to extract the information* 
                 The DICOM standard is complex and there are number of different tools to work with DICOM files, I have used the pydicom package, which is a package that can be imported towards working with images in python.

**File Description**
        Each patient ID has an associated directory of DICOM files and is in the header of the file along with the patient name, the exact number of images per person varies with every file.
stage1.7z - contains all images for the first stage of the competition, including both the training and test set. 
stage1_labels.csv - contains the cancer ground truth for the stage 1 training set images.
data_password.txt - contains the decryption key for the image files.

**3D CNN Computation** 
This section consists of the steps involved taking the processed data and using it to determine the accuracy of the model, to predict whether a person could be affected with cancer or not. To understand the composition of what goes into creating a Convolutional neural network and the layers that are used to process the data, a summary of every layer and its functions that we have used in the project is provided, the summary is an amalgamation of inferences that we have achieved based on various research papers whose links are also provided (references).
We should input the 3D image that we have created in the pre-processing section and set the various layers with values, there is no hard and fast rule that values are fixed, hence we had to test the layers’ multiple times with different values and arrive at a set of values that were optimal.
So, when we train the model, we send in a part of the data (majority) and keep the rest of it for validation purposes to check is the model is well trained. We had 1595 images in total, and we provided 1495 images as input for training the model and 100 images for validation.
The idea here is to complete training the model with 1495 images and then pass the 100 validation images to test the model to provide an accuracy as to how well the model is trained and can predict accurately. In short, we are providing the test data on the trained data model to identify the accuracy (final accuracy).
We also had setup the number of epochs to run, batch size of the input data specifying how many images are to processed at once, input size of the image was also specified as 50x50x20, it is very small compared to the original size of the image, but our systems could accommodate only such capacity, this in turn caused a drop in the quality and quantity of the dataset.
This drop in metrics lead us to observe that we are not using enough information for the model to train on and hence affect the accuracy of the model. After running the model for 100 epochs, we found that the percentage of accuracy was stagnant at the 83rd epoch at around 72%, which is the maximum accuracy we could achieve, we could not achieve more due to lack of more data causing the model to overfit, leading to lower accuracy. Before identifying the final accuracy, we had to mix and match the layers in between to get the optimal layer composition, i.e., At first we had set 3 layers and we had achieved an accuracy of 54% with just 3 epochs run, then we increased the number of epochs and observed that the accuracy increased, we kept increasing the layers and finally reached 6 layers which was optimal and 100 epochs was just fine to get the maximum accuracy.

**Computation of confusion matrix and cancer prediction** 
The confusion matrix is a way to determine whether the model is good or not. Here we have done the confusion matrix for predicted labels vs actual labels. The 0’s represent no cancer and the 1’s represent cancer. Ideally the false positives and the false negatives value should be low.

**Dataset is not enough – overfitting occurs** 
The total dataset size is 1595 out of which 1107 patients do not have cancer, 390 patients have cancer and 98 patients do not have labels. The dataset for number of patients having cancer is very low. This is the reason why overfitting occurs and cancer predictions are not that accurate. One way to overcome this problem is by adding more cancer patient datasets which is currently not available. Try adding LUNA16 dataset and also image rotation and zooming to the existing dataset.

**Concluding Remarks**
The idea that we have presented is at the very beginning of its deployment phase and thus we wish to extend our project by implementing a model that can make valuable predictions based on the reports specified in the datasets and achieve a higher value of accuracy by using Microsoft ResNet approach to build a network, to provide much higher learning rate of the nodes increasing the performance of the model and minimizing the losses effectively.





In [ ]:
import numpy as np
import pandas as pd
import dicom
import os
import matplotlib.pyplot as plt
import cv2
import math

dataDirectory = 'Lung_Cancer/stage1/stage1/'
lungPatients = os.listdir(dataDirectory)
labels = pd.read_csv('Lung_Cancer/stage1_labels/stage1_labels.csv', index_col=0)
size = 50
NoSlices = 20


def chunks(l, n):
    count = 0
    for i in range(0, len(l), n):
        if (count < NoSlices):
            yield l[i:i + n]
            count = count + 1


def mean(l):
    return sum(l) / len(l)


def dataProcessing(patient, labels_df, size=50, noslices=20, visualize=False):
    label = labels_df.get_value(patient, 'cancer')
    path = dataDirectory + patient
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key=lambda x: int(x.ImagePositionPatient[2]))

    new_slices = []
    slices = [cv2.resize(np.array(each_slice.pixel_array), (size, size)) for each_slice in slices]

    chunk_sizes = math.floor(len(slices) / noslices)
    for slice_chunk in chunks(slices, chunk_sizes):
        slice_chunk = list(map(mean, zip(*slice_chunk)))
        new_slices.append(slice_chunk)

    if label == 1:
        label = np.array([0, 1])
    elif label == 0:
        label = np.array([1, 0])
    return np.array(new_slices), label


imageData = []
for num, patient in enumerate(lungPatients):
    if num % 100 == 0:
        print('Saved -', num)
    try:
        img_data, label = dataProcessing(patient, labels, size=size, noslices=NoSlices)
        imageData.append([img_data, label,patient])
    except KeyError as e:
        print('Data is unlabeled')

np.save('imageDataNew-{}-{}-{}.npy'.format(size, size, NoSlices), imageData)

Saved - 0
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Saved - 100
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Saved - 200
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Saved - 300
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Saved - 400
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Saved - 500
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Saved - 600
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Saved - 700
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Saved - 800
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Saved - 900
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Saved - 1000
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Saved - 1100
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Saved - 1200
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Saved - 1300
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Saved - 1400
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Saved - 1500
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled
Data is unlabeled

In [ ]:
import tensorflow as tf
import pandas as pd
import tflearn
from tflearn.layers.conv import conv_3d, max_pool_3d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

imageData = np.load('imageDataNew-50-50-20.npy')
trainingData = imageData[0:800]
validationData = imageData[-200:-100]

x = tf.placeholder('float')
y = tf.placeholder('float')
size = 50
keep_rate = 0.8
NoSlices = 20


def convolution3d(x, W):
    return tf.nn.conv3d(x, W, strides=[1, 1, 1, 1, 1], padding='SAME')


def maxpooling3d(x):
    return tf.nn.max_pool3d(x, ksize=[1, 2, 2, 2, 1], strides=[1, 2, 2, 2, 1], padding='SAME')


def cnn(x):
    x = tf.reshape(x, shape=[-1, size, size, NoSlices, 1])
    convolution1 = tf.nn.relu(
        convolution3d(x, tf.Variable(tf.random_normal([3, 3, 3, 1, 32]))) + tf.Variable(tf.random_normal([32])))
    convolution1 = maxpooling3d(convolution1)
    convolution2 = tf.nn.relu(
        convolution3d(convolution1, tf.Variable(tf.random_normal([3, 3, 3, 32, 64]))) + tf.Variable(
            tf.random_normal([64])))
    convolution2 = maxpooling3d(convolution2)
    convolution3 = tf.nn.relu(
        convolution3d(convolution2, tf.Variable(tf.random_normal([3, 3, 3, 64, 128]))) + tf.Variable(
            tf.random_normal([128])))
    convolution3 = maxpooling3d(convolution3)
    convolution4 = tf.nn.relu(
        convolution3d(convolution3, tf.Variable(tf.random_normal([3, 3, 3, 128, 256]))) + tf.Variable(
            tf.random_normal([256])))
    convolution4 = maxpooling3d(convolution4)
    convolution5 = tf.nn.relu(
        convolution3d(convolution4, tf.Variable(tf.random_normal([3, 3, 3, 256, 512]))) + tf.Variable(
            tf.random_normal([512])))
    convolution5 = maxpooling3d(convolution4)
    fullyconnected = tf.reshape(convolution5, [-1, 1024])
    fullyconnected = tf.nn.relu(
        tf.matmul(fullyconnected, tf.Variable(tf.random_normal([1024, 1024]))) + tf.Variable(tf.random_normal([1024])))
    fullyconnected = tf.nn.dropout(fullyconnected, keep_rate)
    output = tf.matmul(fullyconnected, tf.Variable(tf.random_normal([1024, 2]))) + tf.Variable(tf.random_normal([2]))
    return output


def network(x):
    prediction = cnn(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(cost)
    epochs = 10
    with tf.Session() as session:
        session.run(tf.global_variables_initializer())
        for epoch in range(epochs):
            epoch_loss = 0
            for data in trainingData:
                try:
                    X = data[0]
                    Y = data[1]
                    _, c = session.run([optimizer, cost], feed_dict={x: X, y: Y})
                    epoch_loss += c
                except Exception as e:
                    pass

            correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
           # if tf.argmax(prediction, 1) == 0:
            accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
            print('Epoch', epoch + 1, 'completed out of', epochs, 'loss:', epoch_loss)
            # print('Correct:',correct.eval({x:[i[0] for i in validationData], y:[i[1] for i in validationData]}))
            print('Accuracy:', accuracy.eval({x: [i[0] for i in validationData], y: [i[1] for i in validationData]}))
        print('Final Accuracy:', accuracy.eval({x: [i[0] for i in validationData], y: [i[1] for i in validationData]}))
        patients = []
        actual = []
        predicted = []

        finalprediction = tf.argmax(prediction, 1)
        actualprediction = tf.argmax(y, 1)
        for i in range(len(validationData)):
            patients.append(validationData[i][2])
        for i in finalprediction.eval({x: [i[0] for i in validationData], y: [i[1] for i in validationData]}):
            if(i==1):
                predicted.append("Cancer")
            else:
                predicted.append("No Cancer")
        for i in actualprediction.eval({x: [i[0] for i in validationData], y: [i[1] for i in validationData]}):
            if(i==1):
                actual.append("Cancer")
            else:
                actual.append("No Cancer")
        for i in range(len(patients)):
            print("Patient: ",patients[i])
            print("Actual: ", actual[i])
            print("Predcited: ", predicted[i])

        from sklearn.metrics import confusion_matrix
        y_actual = pd.Series(
            (actualprediction.eval({x: [i[0] for i in validationData], y: [i[1] for i in validationData]})),
            name='Actual')
        y_predicted = pd.Series(
            (finalprediction.eval({x: [i[0] for i in validationData], y: [i[1] for i in validationData]})),
            name='Predicted')
        df_confusion = pd.crosstab(y_actual, y_predicted)
        print(df_confusion)

        def plot_confusion_matrix(df_confusion, title='Confusion matrix', cmap=plt.cm.gray_r):\
            
            plt.matshow(df_confusion, cmap=cmap)
            plt.colorbar()
            tick_marks = np.arange(len(df_confusion.columns))
            plt.xticks(tick_marks, df_confusion.columns, rotation=45)
            plt.yticks(tick_marks, df_confusion.index)
            plt.ylabel(df_confusion.index.name)
            plt.xlabel(df_confusion.columns.name)
            plt.show()
        plot_confusion_matrix(df_confusion)
network(x)

Epoch 1 completed out of 10 loss: 3.88795779023e+13
Accuracy: 0.7
Epoch 2 completed out of 10 loss: 1.24509060778e+13
Accuracy: 0.6
Epoch 3 completed out of 10 loss: 6.06095143603e+12
Accuracy: 0.67
Epoch 4 completed out of 10 loss: 3.88584119686e+12
Accuracy: 0.71
Epoch 5 completed out of 10 loss: 3.01922889866e+12
Accuracy: 0.58
Epoch 6 completed out of 10 loss: 1.83550548122e+12
Accuracy: 0.48
Epoch 7 completed out of 10 loss: 1.58098134931e+12
Accuracy: 0.61
Epoch 8 completed out of 10 loss: 1.02996348973e+12
Accuracy: 0.67
Epoch 9 completed out of 10 loss: 845322210816.0
Accuracy: 0.52
Epoch 10 completed out of 10 loss: 613924536640.0
Accuracy: 0.6
Final Accuracy: 0.61
Patient:  dc1ecce5e7f8a4be9082cb5650fa62bd
Actual:  No Cancer
Predcited:  No Cancer
Patient:  dc5cd907d9de1ed0609832f5bf1fc6e2
Actual:  No Cancer
Predcited:  Cancer
Patient:  dc66d11755fd073a59743d0df6b62ee2
Actual:  No Cancer
Predcited:  Cancer
Patient:  dc9854bcdcc71b690d9806438009001d
Actual:  Cancer
Predcited:  No Cancer
Patient:  dcb426dd025b609489c8f520d6d644b7
Actual:  No Cancer
Predcited:  No Cancer
Patient:  dcde02d4757bb845376fa6dbb0351df6
Actual:  No Cancer
Predcited:  No Cancer
Patient:  dcdf0b64b314e08e8f71f3bec9ecb080
Actual:  No Cancer
Predcited:  No Cancer
Patient:  dcf5fd36b9fff9183f63df286bf8eef9
Actual:  No Cancer
Predcited:  No Cancer
Patient:  dcf75f484b2d2712e5033ba61fd6e2a0
Actual:  No Cancer
Predcited:  Cancer
Patient:  dd281294b34eb6deb76ef9f38169d50e
Actual:  No Cancer
Predcited:  Cancer
Patient:  dd571c3949cdae0b59fc0542bb23e06a
Actual:  No Cancer
Predcited:  Cancer
Patient:  dd5764803d51c71a27707d9db8c84aac
Actual:  No Cancer
Predcited:  No Cancer
Patient:  de04fbf5e6c2389f0d039398bdcda971
Actual:  No Cancer
Predcited:  No Cancer
Patient:  de4d3724030397e71d2ac2ab16df5fba
Actual:  No Cancer
Predcited:  Cancer
Patient:  de635c85f320131ee743733bb04e65b9
Actual:  No Cancer
Predcited:  Cancer
Patient:  de881c07adc8d53e52391fac066ccb9f
Actual:  No Cancer
Predcited:  No Cancer
Patient:  de9f65a7a70b73ce2ffef1e4a2613eee
Actual:  No Cancer
Predcited:  Cancer
Patient:  df015da931ad5312ee7b24b201b67478
Actual:  Cancer
Predcited:  No Cancer
Patient:  df1354de25723c9a55e1241d4c40ffe2
Actual:  No Cancer
Predcited:  No Cancer
Patient:  df54dc42705decd3f75ec8fd8040e76e
Actual:  No Cancer
Predcited:  No Cancer
Patient:  df75d5a21b4289e8df6e2d0e135ac48f
Actual:  No Cancer
Predcited:  No Cancer
Patient:  df761dd787bfc439890740ccce934f36
Actual:  Cancer
Predcited:  Cancer
Patient:  df8614fd49a196123c5b88584dd5dd65
Actual:  No Cancer
Predcited:  No Cancer
Patient:  e00832e96709eb85f8e0e608ca02c2b5
Actual:  Cancer
Predcited:  No Cancer
Patient:  e10c2b829c39d4a500c09caf04d461a1
Actual:  Cancer
Predcited:  No Cancer
Patient:  e127111e994be5f79bb0cea52c9d563e
Actual:  No Cancer
Predcited:  No Cancer
Patient:  e129305f6d074d08cd2de0ebdfeaa576
Actual:  No Cancer
Predcited:  No Cancer
Patient:  e1584618a0c72f124fe618e1ed9b3e55
Actual:  No Cancer
Predcited:  Cancer
Patient:  e163325ccf00afde107c80dfce2bce80
Actual:  No Cancer
Predcited:  No Cancer
Patient:  e188bdeea72bb41d980dc2556dc8aafa
Actual:  No Cancer
Predcited:  No Cancer
Patient:  e1c92d3f85a37bd8bb963345b6d66e03
Actual:  No Cancer
Predcited:  No Cancer
Patient:  e1e47812eecd80466cf7f5b0160de446
Actual:  No Cancer
Predcited:  No Cancer
Patient:  e1f3a01e73d706b7e9c30c0a17a4c0b5
Actual:  No Cancer
Predcited:  No Cancer
Patient:  e2a7eaebd0830061e77690aa48f11936
Actual:  No Cancer
Predcited:  No Cancer
Patient:  e2b7fe7fbb002029640c0e65e3051888
Actual:  Cancer
Predcited:  Cancer
Patient:  e2bcbfe1ab0f9ddc5d6234f819cd5df5
Actual:  No Cancer
Predcited:  No Cancer
Patient:  e2ea2f046495909ff89e18e05f710fee
Actual:  No Cancer
Predcited:  No Cancer
Patient:  e3034ac9c2799b9a9cee2111593d9853
Actual:  No Cancer
Predcited:  No Cancer
Patient:  e3423505ef6b43f03c5d7bde52a5a78c
Actual:  Cancer
Predcited:  No Cancer
Patient:  e38789c5eabb3005bfb82a5298055ba0
Actual:  Cancer
Predcited:  Cancer
Patient:  e3a9a6f8d21c6c459728066bcf18c615
Actual:  No Cancer
Predcited:  No Cancer
Patient:  e3e518324e1a85b85f15d9127ed9ea89
Actual:  No Cancer
Predcited:  No Cancer
Patient:  e414153d0e52f70cbe27c129911445a0
Actual:  No Cancer
Predcited:  No Cancer
Patient:  e42815372aa308f5943847ad06f529de
Actual:  No Cancer
Predcited:  No Cancer
Patient:  e43afa905c8e279f818b2d5104f6762b
Actual:  Cancer
Predcited:  No Cancer
Patient:  e4421d2d5318845c1cccbc6fa308a96e
Actual:  No Cancer
Predcited:  No Cancer
Patient:  e4436b5914162ff7efea2bdfb71c19ae
Actual:  No Cancer
Predcited:  Cancer
Patient:  e46973b13a7a6f421430d81fc1dda970
Actual:  No Cancer
Predcited:  No Cancer
Patient:  e4a87107f94e4a8e32b735d18cef1137
Actual:  No Cancer
Predcited:  No Cancer
Patient:  e4ff18b33b7110a64f497e177102f23d
Actual:  No Cancer
Predcited:  No Cancer
Patient:  e537c91cdfa97d20a39df7ef04a52570
Actual:  Cancer
Predcited:  No Cancer
Patient:  e5438d842118e579a340a78f3c5775cc
Actual:  No Cancer
Predcited:  No Cancer
Patient:  e54b574a7e7c650edc224cbdede9e675
Actual:  Cancer
Predcited:  No Cancer
Patient:  e56b9f25a47a42f4ae4085005c46109c
Actual:  Cancer
Predcited:  No Cancer
Patient:  e572e978c2b50aca781e6302937e5b13
Actual:  Cancer
Predcited:  Cancer
Patient:  e58b78dc31d80a50285816f4ecd661e3
Actual:  Cancer
Predcited:  No Cancer
Patient:  e58cc57cab8a1738041b72b156fedc56
Actual:  No Cancer
Predcited:  No Cancer
Patient:  e5c68cfa0f33540da3098800f0daae2c
Actual:  Cancer
Predcited:  No Cancer
Patient:  e5cf847e616cc2fe94816ffa547d2614
Actual:  Cancer
Predcited:  No Cancer
Patient:  e608c0e6cf3adf3c9939593a3c322ef7
Actual:  No Cancer
Predcited:  No Cancer
Patient:  e6214ef879c6d01ae598161e50e23c0c
Actual:  No Cancer
Predcited:  No Cancer
Patient:  e63f43056330bc418a11208aa3a9e7f0
Actual:  No Cancer
Predcited:  No Cancer
Patient:  e659f6517c4df17e86d4d87181396ea6
Actual:  Cancer
Predcited:  No Cancer
Patient:  e67bc6cd24a71a486b626592d591a2da
Actual:  No Cancer
Predcited:  No Cancer
Patient:  e6b3e750c6c7a70ca512d77defcfe615
Actual:  Cancer
Predcited:  No Cancer
Patient:  e6d4a747235bfcc1feac759571c8485c
Actual:  No Cancer
Predcited:  No Cancer
Patient:  e6d8b2631843a24e6761f2723ea30788
Actual:  No Cancer
Predcited:  No Cancer
Patient:  e6f4757b8f315f31559c5c256cb8dead
Actual:  No Cancer
Predcited:  No Cancer
Patient:  e709901da9ba15a95d4a29906edc01dd
Actual:  Cancer
Predcited:  No Cancer
Patient:  e787e5fd289a9f1f6bba31569b7ad384
Actual:  No Cancer
Predcited:  No Cancer
Patient:  e79f52e833ccca893509f0fdeeb26e9f
Actual:  No Cancer
Predcited:  No Cancer
Patient:  e7adb2e4409683b9490e34b6b3604d9e
Actual:  No Cancer
Predcited:  Cancer
Patient:  e7cb27a5362a7098e1437bfb1224d2dc
Actual:  No Cancer
Predcited:  No Cancer
Patient:  e7d76f0723911280b64f0f83a4990c97
Actual:  No Cancer
Predcited:  No Cancer
Patient:  e858263b89f0bb57597bcff325eaeecf
Actual:  No Cancer
Predcited:  No Cancer
Patient:  e8be143b9f5e352f71043b24f79f5a17
Actual:  No Cancer
Predcited:  No Cancer
Patient:  e8eb842ee04bbad407f85fe671f24d4f
Actual:  Cancer
Predcited:  No Cancer
Patient:  e92a2ed80510513497d5252b001cfa3e
Actual:  No Cancer
Predcited:  No Cancer
Patient:  e977737394cee9abb19ad07310aae8eb
Actual:  No Cancer
Predcited:  No Cancer
Patient:  e9ccf1ce85c39779fafb9ec703c71555
Actual:  Cancer
Predcited:  No Cancer
Patient:  ea7373271a2441b5864df2053c0f5c3e
Actual:  Cancer
Predcited:  No Cancer
Patient:  eacb38abacf1214f3b456b6c9fa78697
Actual:  No Cancer
Predcited:  No Cancer
Patient:  ead64f9269f2200e1d439960a1e069b4
Actual:  No Cancer
Predcited:  No Cancer
Patient:  eaf753dc137e12fd06e96d27f3111043
Actual:  Cancer
Predcited:  No Cancer
Patient:  eb008af181f3791fdce2376cf4773733
Actual:  Cancer
Predcited:  Cancer
Patient:  eb8d5136918d6859ca3cc3abafe369ac
Actual:  No Cancer
Predcited:  No Cancer
Patient:  eba18d04b18084ef64be8f22bb7905ca
Actual:  No Cancer
Predcited:  No Cancer
Patient:  eba4bfb93928d424ff21b5be96b5c09b
Actual:  No Cancer
Predcited:  No Cancer
Patient:  ebd601d40a18634b100c92e7db39f585
Actual:  Cancer
Predcited:  No Cancer
Patient:  ed0f3c1619b2becec76ba5df66e1ea56
Actual:  Cancer
Predcited:  No Cancer
Patient:  ed49b57854f5580658fb3510676e03dd
Actual:  Cancer
Predcited:  Cancer
Patient:  ed83b655a1bbad40a782ad13cf27ce8f
Actual:  No Cancer
Predcited:  No Cancer
Patient:  eda58f4918c4b506cd156702bf8a56a3
Actual:  No Cancer
Predcited:  No Cancer
Patient:  edad1a7e85b5443e0ae9e654d2adbcba
Actual:  No Cancer
Predcited:  No Cancer
Patient:  edae2e1edd1217d0c9e20eff2a7b2dd8
Actual:  No Cancer
Predcited:  No Cancer
Patient:  edbf53a8478049de1494b213fdf942e6
Actual:  No Cancer
Predcited:  Cancer
Patient:  ee71210fa398cbb080f6c537a503e806
Actual:  No Cancer
Predcited:  Cancer
Patient:  ee88217bee233a3bfc971b450e3d8b85
Actual:  No Cancer
Predcited:  No Cancer
Patient:  ee984e8fba88691aac4992fbb14f6e97
Actual:  No Cancer
Predcited:  Cancer
Patient:  ee9c580272cd02741df7299892602ac7
Actual:  No Cancer
Predcited:  No Cancer
Predicted   0   1
Actual           
0          56  17
1          18   9